Initializations:

In [1]:
%reload_ext snakeviz
%matplotlib inline
%run SBRG2.py

#Core
##Pauli Algebra
Pauli matrix class. A Pauli matrix is denoted as
$$\sigma=\mathrm{i}^{x\cdot z}\prod_{i\in Xs}X_i\prod_{i\in Zs}Z_i=\mathrm{i}^{x\cdot z}\prod_{i}X_i^{x_i}Z_i^{z_i},$$
where $x_i=\delta(i\in Xs)$, $z_i=\delta(i\in Zs)$ are vectors of 0,1, and $X_i$, $Z_i$ are the Pauli matrix on site $i$.

Pauli matrices can be created by `mkMat`
- from Xs, Zs sets: `mkMat(Xs, Zs)`,
- or from dict of indices: `mkMat({i:mu, ...})`,
- or from list of indices: `mkMat([m0, mu1, ...])`.

In [252]:
[mkMat({0,1},{1,2}), mkMat({0:1,1:2,2:3}), mkMat([1,2,3])]

[<Xs:[0, 1] Zs:[1, 2]>, <Xs:[0, 1] Zs:[1, 2]>, <Xs:[0, 1] Zs:[1, 2]>]

`Mat` is hashable and can be compared.

In [248]:
print(hash(mkMat()))
mkMat() == mkMat([])

6795291966596493067


True

The commutativity of two Pauli operators can be checked by `is_commute`

In [249]:
is_commute(mkMat([0,1,0]),mkMat([1,2,3]))

False

Merge two Pauli operators by indices. Coefficients are not calculated here. The coefficient can be restored from the commutation relations later.

In [250]:
pdot(mkMat([0,1,0]),mkMat([1,2,3]))

<Xs:[0] Zs:[1, 2]>

###Pauli Monomial
A term of Pauli operator `Term.mat` with a constant multiple `Term.val`.

In [255]:
[Term(), Term(mkMat([1,2])),Term(mkMat([1,2]),-3.2)]

[1.0 <Xs:[] Zs:[]>, 1.0 <Xs:[0, 1] Zs:[1]>, -3.2 <Xs:[0, 1] Zs:[1]>]

Consider $\sigma_A=\mathrm{i}^{x_A\cdot z_A} X_A Z_A$, $\sigma_B=\mathrm{i}^{x_B\cdot z_B} X_B Z_B$, then the product reads:
$$\sigma_A\sigma_B=\mathrm{i}^{x_A\cdot z_A+x_B\cdot z_B+2 z_A\cdot x_B}\quad (X_A X_B) (Z_A Z_B)=\mathrm{i}^{x_A\cdot z_A+x_B\cdot z_B-x_C\cdot z_C+2 z_A\cdot x_B}\quad \mathrm{i}^{x_C\cdot z_C}X_C Z_C=\mathrm{i}^{n^{C}_{AB}}\; \sigma_C,$$
where $X_C=X_AX_B$, $Z_C=Z_AZ_B$ and $z_C = (z_A+z_B)\mod 2$. The power of $\mathrm{i}$ is
$$n^{C}_{AB}=x_A\cdot z_A+x_B\cdot z_B-x_C\cdot z_C+2 z_A\cdot x_B$$

In [256]:
dot(Term(mkMat([1,2])),Term(mkMat([3,3])))

1.0 <Xs:[0, 1] Zs:[0]>

In [257]:
idot(Term(mkMat([2,0])),Term(mkMat([3,3])))

-1.0 <Xs:[0] Zs:[1]>

##Hamiltonian
Hamiltonian `Ham` is a sum of Pauli monomial terms. It can be created from a list of terms.

In [258]:
H = Ham(
    [Term(mkMat([0,1,1]),0.5),
     Term(mkMat([3,0,0]),0.2),
     Term(mkMat([0,3,3]),0.8),
     Term(mkMat([1,3,0]),1.0)])
H

[1.0 <Xs:[0] Zs:[1]>, 0.8 <Xs:[] Zs:[1, 2]>, 0.5 <Xs:[1, 2] Zs:[]>, 0.2 <Xs:[] Zs:[0]>]

In [259]:
H.forward([Term(mkMat([0],[0,1]),-1.)])
H

[1.0 <Xs:[] Zs:[0]>, 0.8 <Xs:[] Zs:[1, 2]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>, -0.2 <Xs:[0] Zs:[1]>]

In [260]:
H.extend([Term(mkMat([1,3,0]),0.1)])
H

[1.0 <Xs:[] Zs:[0]>, 0.8 <Xs:[] Zs:[1, 2]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>, -0.1 <Xs:[0] Zs:[1]>]

In [5]:
H.remove(H.terms[1])
H

[1.0 <Xs:[] Zs:[0]>, -0.1 <Xs:[0] Zs:[1]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>]

In [261]:
print([H.mats[term.mat] == term for term in H])
print([[i in (term.mat.Xs | term.mat.Zs) for term in terms] for (i, terms) in H.imap.items()])
print([term.pos for term in H])

[True, True, True, True]
[[True, True, True], [True, True, True], [True, True]]
[0, 1, 2, 3]


## SBRG

Profiling

In [64]:
system = SBRG(TFIsing(256,J=4.,K=1.,h=1.,alpha=0.5))
% snakeviz system.run()

 
*** Profile stats marshalled to file '/var/folders/tl/lwpcq5qj049ftcj7pvhkzv_h0000gn/T/tmpnzp9nz'. 


Benchmark with SBRG1

In [5]:
random.seed(2)
model = TFIsing(8,J=2.,K=1.,h=1.,alpha=1.)
system = SBRG(model)
system.tol = 0.
system.max_rate = 1000.
system.run()

###Energy Coefficients

In [37]:
from IPython.display import clear_output
def span(mat,L): # get the span of Zs of a Mat
    mus = list(mat.Zs)
    mus.sort()
    diff = [snd-fst for fst, snd in zip(mus, mus[1:] + [mus[0]+L])]
    i = diff.index(max(diff))
    mu0 = mus[(i+1)%len(mus)]
    mu1 = mus[i]
    return (mu1-mu0)%L
fname = 'ECdat_J2K1h1_a10'
realization = 50
L = 256
try:
    ECdat = load(fname)
except:
    ECdat = []
for i in range(realization):
    clear_output(wait=True)
    print(i)
    system = SBRG(TFIsing(L,J=2.,K=1.,h=1.,alpha=1.)).run()
    ECdat.extend([[span(term.mat,L),len(term.mat.Zs),term.val] for term in system.Heff])
dump(fname, ECdat)
export(fname, ECdat)

49


###Statistics of Energy Coefficients

In [42]:
random.seed(2)
system = SBRG(TFIsing(14,J=2.,K=1.,h=1.,alpha=1.)).run()
from itertools import chain, combinations
def subsets(iterable):
    s = list(iterable)
    return map(set, chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))
emap = [[s,0] for s in subsets(range(0,system.size))];
for term in system.Heff:
    Zs = term.mat.Zs
    v = term.val
    for em in emap:
        if len(Zs & em[0])%2 == 1:
            em[1] += v
e0 = sum(term.val for term in system.Heff)
es = [e0-2*em[1] for em in emap]
export('Es_J2K1h1_a10_14',es)

###Wave Function Overlap
Benchmark with ED fidelity.

In [75]:
def tolst(term):
    return [list(term.mat.Xs),list(term.mat.Zs),term.val]
random.seed(3)
system = SBRG(TFIsing(8,J=1.,K=1.,h=1.,alpha=0.2))
RGdat = []
while system.phybits:
    (H0, Rs, offdiag) = system.nextstep()
    RGdat.append([tolst(H0),[tolst(R) for R in Rs],[tolst(S) for S in offdiag]])
Hbdy = [tolst(term) for term in system.Hbdy]
Heff = [tolst(term) for term in system.Heff]
export('WFO_J1K1h1_a02_8',{'bits':system.size,'RGdat': RGdat,'Hbdy':Hbdy,'Heff':Heff})

###RG Flow
Collect histogram of Hamiltonian coefficients (in logrithmic scale).

In [293]:
import math
import numpy as np
realization = 512
bins = [x/10 for x in range(-1,501)]
hdic = {}
for i in range(realization):
    system = SBRG(TFIsing(512,J=1.,K=0.,h=1.,alpha_J=1.,alpha_K=1.,alpha_h=1.))
    n = len(system.phybits)
    while n>=4:
        hs = [-math.log(abs(term.val)) for term in system.H]
        hist, bin_edges = np.histogram(hs,bins)
        try:
            hdic[n] += hist
        except:
            hdic[n] = hist
        system.flow(n/2)
        n = len(system.phybits)
hdic = {n: hist.tolist() for n, hist in hdic.items()}
export('RG_J1K0h1_a101010_512', {'bins':bins,'hdic':hdic})

However the Hamiltonian coefficient distribution does not directly reflect how good is the perturbation. The small parameter of perturbation is off-diagonal to diagonal ratio (ODR). The following collects this statistics.

In [2]:
from IPython.display import clear_output
import math
import numpy as np
def logODR(sys):
    if not sys.H: # if H is empty
        return[]
    H0 = sys.H.terms[0]
    E0 = abs(H0.val)
    if not E0: # if E0 = 0, return
        return []
    Rs, pbit = sys.findRs(H0.mat)
    sys.H.forward(Rs)
    lr = [-math.log(abs(term.val/E0)) for term in sys.H.imap[pbit] if pbit in term.mat.Xs]
    sys.H.backward(Rs)
    return lr
realization = 2000
bins = [x/10 for x in range(-1,301)]
fname = 'ODR_J1K1h1_a20_512'
try:
    hdic = load(fname)
except:
    hdic = {}
for i in range(realization):
    clear_output(wait=True)
    print(i)
    system = SBRG(TFIsing(512,J=1.,K=1.,h=1.,alpha=2.0))
    n = len(system.phybits)
    while n>=4:
        hs = logODR(system)
        hist, bin_edges = np.histogram(hs,bins)
        try:
            hdic[n] += hist
        except:
            hdic[n] = hist
        system.flow(n/2)
        n = len(system.phybits)
dump(fname, hdic)
hdic = {n: hist.tolist() for n, hist in hdic.items()}
export(fname, {'bins':bins,'hdic':hdic})

1999


In [ ]:
dump(fname, hdic)
hdic = {n: hist.tolist() for n, hist in hdic.items()}
export(fname, {'bins':bins,'hdic':hdic})

After discussion with Kevin, I realize that it is better to use the $g$ statistics.
$$g=\frac{||\Sigma||}{||H_0||}$$

In [33]:
from IPython.display import clear_output
import math
import numpy as np
system = SBRG(TFIsing(512,J=1.,K=0.,h=2.,alpha=1.))
hs = []
while system.phybits:
    system.nextstep()
    hs.append(system.ODR)

###Entanglement

Half-system-size entanglement entropy.

In [157]:
a = 1.0
N = 256
filename = 'PD_a10_256'
def EE(J0, K0, h0, sampling = 1):
    try:
        entropy_rec = load(filename)
    except:
        entropy_rec = {}
    samples = []
    L = int(N/2)
    for s in range(sampling):
        system = SBRG(TFIsing(N,J=J0,K=K0,h=h0,alpha=a))
        system.max_rate = 1.8
        system.run()
        S = bipartite_entropy(system)
        print('%3d: %f'%(s+1,S))
        samples.append(S)
    pt = (int(J0),int(K0),int(h0))
    try:
        entropy_rec[pt].extend(samples)
    except:
        entropy_rec[pt] = samples
    dump(filename,entropy_rec)
    export(filename,entropy_rec)
    return

In [ ]:
EE(2,1,1,10)

### Anderson Correlator
First use the random cliford circuit to forward transform the operators to the holographic space. Then two operators are correlated if their product does not involves any flipping of the emergent qubit (such that can have expectation value $\pm1$ in the product state of emergent qubits).

Snapshot of the boundary operators mapped into the bulk:

In [194]:
system = SBRG(XYZ(8, Jx=1,Jy=1,Jz=0,alpha=0.5))
system.run()
for mus in [(1,), (1,1)]:
    print(mus, '----------')
    ops = Ham([Term(mkMat({(i+j)%system.size:mus[i] for i in range(len(mus))}))
               for j in range(system.size)])
    ops.forward(system.RCC)
    for term in ops:
        print(term.mat)

(1,) ----------
<Xs:[0, 2, 7] Zs:[0, 1, 7]>
<Xs:[1, 2, 7] Zs:[0, 1, 7]>
<Xs:[1, 3] Zs:[2, 4, 5, 6]>
<Xs:[3] Zs:[]>
<Xs:[4, 7] Zs:[0, 1, 2, 4, 5, 6, 7]>
<Xs:[4, 7] Zs:[0, 1, 2, 4, 6, 7]>
<Xs:[6, 7] Zs:[0, 1, 2, 4, 5, 6, 7]>
<Xs:[0, 6, 7] Zs:[0, 1, 2, 4, 5, 6]>
(1, 1) ----------
<Xs:[0, 1] Zs:[]>
<Xs:[2, 3, 7] Zs:[0, 1, 2, 4, 5, 6, 7]>
<Xs:[1] Zs:[2, 4, 5, 6]>
<Xs:[3, 4, 7] Zs:[0, 1, 2, 4, 5, 6, 7]>
<Xs:[] Zs:[5]>
<Xs:[4, 6] Zs:[5]>
<Xs:[0] Zs:[7]>
<Xs:[2, 6] Zs:[2, 4, 5, 6, 7]>


Collect Anderson-Edward correlator.

In [191]:
from IPython.display import clear_output
def AEC_update(system, mus, correlator):
    L = system.size
    terms = [Term(mkMat({(i+j)%L:mus[i] for i in range(len(mus))})) for j in range(L)]
    new_corr = system.correlate(terms) # terms will be destroyed
    for d, c in new_corr.items():
        correlator[d] = correlator.get(d,0) + c
realization = 1000
fname = 'AEC_X1Y1Z0_a05_256'
try:
    AEC = load(fname)
except:
    AEC = {mus:{} for mus in [(1,),(2,),(3,),(1,1),(2,2),(3,3)]}
for i in range(realization):
    clear_output(wait=True)
    print(i)
    system = SBRG(XYZ(256, Jx=1,Jy=1,Jz=0,alpha=0.5))
    system.run()
    for mus, correlator in AEC.items():
        AEC_update(system, mus, correlator)
    if i%10 == 0:
        dump(fname, AEC)
        export(fname, AEC)
dump(fname, AEC)
export(fname, AEC)

999


Find the case of long-range ZZ correlation.

In [228]:
for i in range(1000):
    clear_output(wait=True)
    print(i)
    system = SBRG(XYZ(64, Jx=1,Jy=1,Jz=0,alpha=0.5))
    system.run()
    opsX = Ham([Term(mkMat({i:1})) for i in range(system.size)])
    opsY = Ham([Term(mkMat({i:2})) for i in range(system.size)])
    opsZ = Ham([Term(mkMat({i:3})) for i in range(system.size)])
    opsX.forward(system.RCC)
    opsY.forward(system.RCC)
    opsZ.forward(system.RCC)
    corZ = {}
    L = system.size
    for (i,j) in combinations(range(len(opsZ)),2):
        if len(opsZ.terms[i].mat.Xs ^ opsZ.terms[j].mat.Xs) == 0:
            d = int(abs((j - i + L/2)%L - L/2))
            corZ[d] = corZ.get(d,0) + 1
    if any(d>3 for d in corZ):
        print('found!')
        export_Ham('taus',system.taus)
        export('ops',[[i,list(opsX.terms[i].mat.Xs),
                   list(opsY.terms[i].mat.Xs),
                   list(opsZ.terms[i].mat.Xs)] for i in range(system.size)])
        break

11
found!


###Track the Flow of Terms

In [516]:
random.seed(1)
system = SBRG(XYZ(256, Jx=1,Jy=1,Jz=1,alpha_x=0.5,alpha_y=0.5,alpha_z=0.01))
system.tol = 0.
system.max_rate = 1.8

In [521]:
system.flow(8)
#all([len(term.mat.Xs)%2==0 for term in system.H])
len(system.H)

362

In [561]:
from IPython.display import clear_output
import math
import numpy as np
bins = [x/10 for x in range(-70,71)]
def Majorana_cls(cnthist, sys):
# sys - SBRG system, bins - bins of couplings in log scale
    H = deepcopy(sys.H)
    H.backward(sys.RCC)
    lmbcls = {}
    for term in H.terms:
        cnt = Majorana_count(sys.size, term.mat)
        lmb = math.log(-math.log(abs(term.val)))
        try:
            lmbcls[cnt].append(lmb) 
        except:
            lmbcls[cnt] = [lmb]
    for cnt, lmbs in lmbcls.items():
        hist, bin_edgs = np.histogram(lmbs, bins)
        try:
            cnthist[cnt] += hist
        except:
            cnthist[cnt] = hist
    return cnthist
def export_flowdic(fname, flowdic):
    dat = {n: {cnt: hist.tolist() for cnt, hist in cnthist.items()} 
           for n, cnthist in flowdic.items()}
    export(fname, {'bins':bins,'flowdic': dat})
realizations = 0
fname = 'ZZflow_GX2Y2Z4_256'
try:
    flowdic = load(fname)
except:
    flowdic = {}
for i in range(realizations):
    clear_output(wait=True)
    print(i)
    system = SBRG(XYZ(256, Jx=1,Jy=1,Jz=1,alpha_x=0.5,alpha_y=0.5,alpha_z=0.25))
    system.tol = 0.
    system.max_rate = 1.6
    n = len(system.phybits)
    while n >= 4:
        try:
            cnthist = flowdic[n]
        except:
            cnthist = {}
            flowdic[n] = cnthist
        Majorana_cls(cnthist,system)
        system.flow(n/2)
        n = len(system.phybits)
    if i%10 == 0:
        dump(fname, flowdic)
        export_flowdic(fname, flowdic)
dump(fname, flowdic)
export_flowdic(fname, flowdic)

In [432]:
import math
math.log(-math.log(1-1.e-3)), math.log(-math.log(1e-323))

(-6.907255070523716, 6.611700822049034)

In [509]:
system.phybits

{24, 86, 104, 115, 126, 148, 160, 172, 192, 197, 211, 243, 250, 255}

###Truncated Flow

In [32]:
%run SBRG2.py

In [35]:
from IPython.display import clear_output
import math
import numpy as np
bins = [x/10 for x in range(-70,71)]
def Majorana_cls(cnthist, sys):
# sys - SBRG system, bins - bins of couplings in log scale
    H = deepcopy(sys.H)
    H.backward(sys.RCC)
    lmbcls = {}
    for term in H.terms:
        cnt = Majorana_count(sys.size, term)
        lmb = math.log(-math.log(abs(term.val)))
        try:
            lmbcls[cnt].append(lmb) 
        except:
            lmbcls[cnt] = [lmb]
    for cnt, lmbs in lmbcls.items():
        hist, bin_edgs = np.histogram(lmbs, bins)
        try:
            cnthist[cnt] += hist
        except:
            cnthist[cnt] = hist
    return cnthist
def export_flowdic(fname, flowdic):
    dat = {n: {cnt: hist.tolist() for cnt, hist in cnthist.items()} 
           for n, cnthist in flowdic.items()}
    export(fname, {'bins':bins,'flowdic': dat})
realizations = 200
fname = 'ZZflow4_GX2Y2Z10_256'
try:
    flowdic = load(fname)
except:
    flowdic = {}
for i in range(realizations):
    clear_output(wait=True)
    print(i)
    system = SBRG(XYZ(256, Jx=1,Jy=1,Jz=1,alpha_x=0.5,alpha_y=0.5,alpha_z=0.1))
    system.tol = 0.
    system.max_rate = 10.
    n = len(system.phybits)
    while n >= 4:
        try:
            cnthist = flowdic[n]
        except:
            cnthist = {}
            flowdic[n] = cnthist
        Majorana_cls(cnthist,system)
        system.flow(n/2)
        n = len(system.phybits)
    if i%10 == 0:
        dump(fname, flowdic)
        export_flowdic(fname, flowdic)
dump(fname, flowdic)
export_flowdic(fname, flowdic)

199


In [3]:
def short_Maj(L, term):
    Maj = term.Maj
    if len(Maj) > L:
        Maj.symmetric_difference_update(range(2*L))
    return Maj

In [31]:
random.seed(1)
system = SBRG(XYZ(4, Jx=1,Jy=1,Jz=1,alpha_x=0.5,alpha_y=0.5,alpha_z=0.01))
system.tol = 0.
system.max_rate = 10.
system.flow(2)
[(term.val, term.mat, short_Maj(system.size,term)) for term in system.H]

[(0.10237109018699665, <Xs:[0, 3] Zs:[0, 1, 2, 3]>, {0, 3}),
 (-0.048561065818736766, <Xs:[0, 3] Zs:[1, 2]>, {1, 6}),
 (0.051384032277701906, <Xs:[0, 3] Zs:[0, 3]>, {1, 6}),
 (-8.623428181091173e-20, <Xs:[] Zs:[0, 3]>, {0, 1, 3, 6}),
 (-0.0013379393458593113, <Xs:[0, 3] Zs:[]>, {0, 3}),
 (1.711903472123438e-13, <Xs:[0, 3] Zs:[1]>, {1, 4, 6, 7}),
 (1.0618477047348558e-172, <Xs:[] Zs:[0, 2, 3]>, {2, 5}),
 (3.702378092265929e-179, <Xs:[] Zs:[0, 1, 3]>, {4, 7}),
 (-2.6081666747851793e-20, <Xs:[0, 3] Zs:[0, 2, 3]>, {0, 2, 3, 5})]

In [21]:
H0 = system.H.terms[0]
h0 = H0.val
# find Clifford rotations
Rs, pbit = system.findRs(H0.mat)
system.RCC.extend(Rs) # add to RCC
system.H.forward(Rs) # apply to H

In [22]:
[(term.val, term.mat, short_Maj(system.size,term)) for term in system.H]

[(0.5017506077128795, <Xs:[] Zs:[1]>, {2, 5}),
 (0.19105119914083313, <Xs:[2, 3] Zs:[2, 3]>, {4, 7}),
 (-0.28383135024859485, <Xs:[1, 3] Zs:[2]>, {5, 6}),
 (0.10237109018699665, <Xs:[0, 2] Zs:[0, 1, 2]>, {0, 3}),
 (0.19032487003196838, <Xs:[0, 3] Zs:[]>, {0, 7}),
 (-1.5244300274234022e-19, <Xs:[1, 2] Zs:[0, 1, 2]>, {0, 1, 2, 3}),
 (1.7184365401515998e-13, <Xs:[] Zs:[1, 2]>, {2, 3, 4, 5}),
 (-0.0013430452697074653, <Xs:[] Zs:[2]>, {3, 4}),
 (3.4949635048172834e-154, <Xs:[1, 2] Zs:[3]>, {4, 5, 6, 7}),
 (0.08584514805850604, <Xs:[0, 1] Zs:[]>, {1, 2}),
 (0.051384032277701906, <Xs:[0, 3] Zs:[0, 3]>, {1, 6}),
 (4.116205385959462e-167, <Xs:[] Zs:[0, 3]>, {0, 1, 6, 7})]

In [23]:
offdiag = [term for term in system.H.imap[pbit] if pbit in term.mat.Xs]
[(term.val, term.mat, short_Maj(system.size,term)) for term in offdiag]

[(0.08584514805850604, <Xs:[0, 1] Zs:[]>, {1, 2}),
 (-1.5244300274234022e-19, <Xs:[1, 2] Zs:[0, 1, 2]>, {0, 1, 2, 3}),
 (-0.28383135024859485, <Xs:[1, 3] Zs:[2]>, {5, 6}),
 (3.4949635048172834e-154, <Xs:[1, 2] Zs:[3]>, {4, 5, 6, 7})]

In [24]:
pert = system.perturbation(H0, offdiag) # 2nd order perturbation
[(term.val, term.mat, short_Maj(system.size,term)) for term in pert]

[(-0.048561065818736766, <Xs:[0, 3] Zs:[1, 2]>, {1, 6}),
 (-2.6081666747851793e-20, <Xs:[0, 2] Zs:[0, 2]>, {0, 2, 3, 5}),
 (-1.9770383839963363e-154, <Xs:[2, 3] Zs:[1, 2, 3]>, {2, 4, 5, 7}),
 (1.0618477047348558e-172, <Xs:[] Zs:[0, 2, 3]>, {2, 5}),
 (5.979577401963025e-155, <Xs:[0, 2] Zs:[1, 3]>, {1, 4, 6, 7}),
 (8.623428181091173e-20, <Xs:[2, 3] Zs:[0]>, {0, 1, 3, 6}),
 (0.08762283839568757, <Xs:[] Zs:[1]>, set())]

In [25]:
for term in offdiag:
    system.H.remove(term) # remove off-diagonal terms
system.H.extend(pert) # add perturbation to H
[(term.val, term.mat, short_Maj(system.size,term)) for term in system.H]

[(0.5893734461085671, <Xs:[] Zs:[1]>, {2, 5}),
 (0.19105119914083313, <Xs:[2, 3] Zs:[2, 3]>, {4, 7}),
 (0.051384032277701906, <Xs:[0, 3] Zs:[0, 3]>, {1, 6}),
 (0.10237109018699665, <Xs:[0, 2] Zs:[0, 1, 2]>, {0, 3}),
 (0.19032487003196838, <Xs:[0, 3] Zs:[]>, {0, 7}),
 (5.979577401963025e-155, <Xs:[0, 2] Zs:[1, 3]>, {1, 4, 6, 7}),
 (1.7184365401515998e-13, <Xs:[] Zs:[1, 2]>, {2, 3, 4, 5}),
 (-0.0013430452697074653, <Xs:[] Zs:[2]>, {3, 4}),
 (-0.048561065818736766, <Xs:[0, 3] Zs:[1, 2]>, {1, 6}),
 (-2.6081666747851793e-20, <Xs:[0, 2] Zs:[0, 2]>, {0, 2, 3, 5}),
 (-1.9770383839963363e-154, <Xs:[2, 3] Zs:[1, 2, 3]>, {2, 4, 5, 7}),
 (1.0618477047348558e-172, <Xs:[] Zs:[0, 2, 3]>, {2, 5}),
 (4.116205385959462e-167, <Xs:[] Zs:[0, 3]>, {0, 1, 6, 7}),
 (8.623428181091173e-20, <Xs:[2, 3] Zs:[0]>, {0, 1, 3, 6})]